### TCLab F: SISO MPC

Single-Input (One heater) Single-Output (One temperature) implementation of Model Predictive Control (MPC) for the temperature control lab. You can extend this lab to include two heaters and two temperature sensors for MPC. Additional details are available in the [Dynamic Optimization course (Lab F)](http://apmonitor.com/do/index.php/Main/TCLabF).

In [ ]:
import pandas as pd
import numpy as np
from gekko import GEKKO
m = GEKKO()

# load data and parse into columns
url = 'http://apmonitor.com/do/uploads/Main/tclab_dyn_data1.txt'
data = pd.read_csv(url)
t = data['Time']
u = data['H1']
y = data['T1']

# system identification
yp,p,K = m.sysid(t,u,y,2,2,pred='model')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(t,y)
plt.plot(t,yp)

In [ ]:
# Build GEKKO ARX model
Q = m.MV(); T = m.CV()
m.arx(p,T,Q)

# set up MV
Q.LOWER = 0; Q.UPPER = 100
Q.STATUS=1;  Q.FSTATUS=0; Q.DCOST = 0.01

# set up CV
T.SPHI  =51;  T.SPLO   = 49
T.STATUS = 1; T.FSTATUS = 1; T.TAU = 30

# steady state initialization
m.options.IMODE = 1
m.solve(disp=False)

# load inputs
tf = 60 # final time (sec)
m.time = np.linspace(0,tf,tf+1)
m.options.NODES = 2

m.options.IMODE = 6
m.solve(disp=False)

In [ ]:
def mpc(Tm):
    T.MEAS = Tm
    m.solve(disp=False)
    return Q.NEWVAL

In [ ]:
import tclab
import time
import numpy as np
import matplotlib.pyplot as plt

n = 180
tm = np.linspace(0,n-1,n)
T1 = np.zeros(n); Q1 = np.zeros(n)

# use tclab.TCLabModel() if no device
lab = tclab.TCLab()
for i in range(n):
    # read temperature
    T1[i] = lab.T1

    # PID control
    Q1[i] = mpc(T1[i]) 
    lab.Q1(Q1[i])

    # print
    if i%50==0:
        print('Time OP PV   SP')
    if i%5==0:
        print(i,round(Q1[i],2), T1[i], 50)
    # wait sample time
    time.sleep(1) # wait 1 sec
lab.close()

In [ ]:
# Create Figure
plt.figure(figsize=(12,8))
plt.subplot(2,1,1)
plt.grid()
plt.plot([0,tm[-1]/60.0],[51,51],'k--',label=r'$T_1$ SPHI')
plt.plot([0,tm[-1]/60.0],[49,49],'k--',label=r'$T_1$ SPLO')
plt.plot(tm/60.0,T1,'r.',label=r'$T_1$ PV')
plt.ylabel(r'Temp ($^oC$)')
plt.legend()
plt.subplot(2,1,2)
plt.grid()
plt.plot(tm/60.0,Q1,'b-',label=r'$Q_1$')
plt.ylabel(r'Heater (%)'); plt.xlabel('Time (min)')
plt.legend()
plt.show()